In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

shape: (5, 13)
┌───────────┬──────────┬───────┬────────┬───┬──────────┬──────────┬───────────┬──────────┐
│ f         ┆ time_idx ┆ dummy ┆ actual ┆ … ┆ a        ┆ b        ┆ y         ┆ y2       │
│ ---       ┆ ---      ┆ ---   ┆ ---    ┆   ┆ ---      ┆ ---      ┆ ---       ┆ ---      │
│ f64       ┆ i64      ┆ str   ┆ i32    ┆   ┆ f64      ┆ f64      ┆ f64       ┆ f64      │
╞═══════════╪══════════╪═══════╪════════╪═══╪══════════╪══════════╪═══════════╪══════════╡
│ 0.0       ┆ 0        ┆ a     ┆ 1      ┆ … ┆ 0.963357 ┆ 0.645792 ┆ -0.087106 ┆ 0.374231 │
│ 0.841471  ┆ 1        ┆ a     ┆ 0      ┆ … ┆ 0.523703 ┆ 0.308267 ┆ 0.315413  ┆ 0.418257 │
│ 0.909297  ┆ 2        ┆ a     ┆ 0      ┆ … ┆ 0.480838 ┆ 0.035734 ┆ -0.436603 ┆ 0.127021 │
│ 0.14112   ┆ 3        ┆ a     ┆ 0      ┆ … ┆ 0.854051 ┆ 0.943042 ┆ 0.055943  ┆ 0.150295 │
│ -0.756802 ┆ 4        ┆ a     ┆ 0      ┆ … ┆ 0.965046 ┆ 0.556006 ┆ -1.355857 ┆ 0.00166  │
└───────────┴──────────┴───────┴────────┴───┴──────────┴──────────┴───────────┴──────────┘

In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

shape: (1, 1)
┌─────┐
│ x1  │
│ --- │
│ f64 │
╞═════╡
│ 0.0 │
└─────┘

In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

shape: (5, 1)
┌──────────────────────┐
│ f                    │
│ ---                  │
│ array[f64, 2]        │
╞══════════════════════╡
│ [1.939505, 0.0]      │
│ [1.939506, 0.000209] │
│ [1.939508, 0.000418] │
│ [1.939512, 0.000627] │
│ [1.939518, 0.000835] │
└──────────────────────┘

In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

shape: (5, 3)
┌────────────┬───────────┬───────────┐
│ f          ┆ a         ┆ b         │
│ ---        ┆ ---       ┆ ---       │
│ f64        ┆ f64       ┆ f64       │
╞════════════╪═══════════╪═══════════╡
│ 1.3944e-15 ┆ -0.963357 ┆ -0.645792 │
│ -0.841471  ┆ -0.523703 ┆ -0.308267 │
│ -0.909297  ┆ -0.480838 ┆ -0.035734 │
│ -0.14112   ┆ -0.854051 ┆ -0.943042 │
│ 0.756802   ┆ -0.001688 ┆ 0.089786  │
└────────────┴───────────┴───────────┘

In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

shape: (1, 1)
┌────────────────────────┐
│ coeffs                 │
│ ---                    │
│ list[f64]              │
╞════════════════════════╡
│ [-0.500734, -0.338584] │
└────────────────────────┘

In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

shape: (1, 2)
┌────────────────────────┬──────────────────────┐
│ target_0               ┆ target_1             │
│ ---                    ┆ ---                  │
│ list[f64]              ┆ list[f64]            │
╞════════════════════════╪══════════════════════╡
│ [-0.500734, -0.338584] ┆ [0.086658, 0.407468] │
└────────────────────────┴──────────────────────┘

In [9]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

shape: (4, 7)
┌──────────┬───────────┬──────────┬──────────────┬───────┬───────────┬───────────┐
│ features ┆ beta      ┆ std_err  ┆ t            ┆ p>|t| ┆ 0.025     ┆ 0.975     │
│ ---      ┆ ---       ┆ ---      ┆ ---          ┆ ---   ┆ ---       ┆ ---       │
│ str      ┆ f64       ┆ f64      ┆ f64          ┆ f64   ┆ f64       ┆ f64       │
╞══════════╪═══════════╪══════════╪══════════════╪═══════╪═══════════╪═══════════╡
│ ln(x1+1) ┆ 0.220087  ┆ 0.001678 ┆ 131.140737   ┆ 0.0   ┆ 0.216797  ┆ 0.223376  │
│ exp(x2)  ┆ 0.174449  ┆ 0.000676 ┆ 258.179775   ┆ 0.0   ┆ 0.173125  ┆ 0.175774  │
│ sin(x3)  ┆ -1.745781 ┆ 0.001346 ┆ -1297.083954 ┆ 0.0   ┆ -1.748419 ┆ -1.743142 │
│ __bias__ ┆ -0.106951 ┆ 0.0015   ┆ -71.292813   ┆ 0.0   ┆ -0.109891 ┆ -0.10401  │
└──────────┴───────────┴──────────┴──────────────┴───────┴───────────┴───────────┘

In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

shape: (10_000, 2)
┌───────┬────────────────────────┐
│ dummy ┆ coeffs                 │
│ ---   ┆ ---                    │
│ str   ┆ list[f64]              │
╞═══════╪════════════════════════╡
│ a     ┆ [-0.479674, -0.344547] │
│ a     ┆ [-0.479674, -0.344547] │
│ a     ┆ [-0.479674, -0.344547] │
│ a     ┆ [-0.479674, -0.344547] │
│ a     ┆ [-0.479674, -0.344547] │
│ …     ┆ …                      │
│ b     ┆ [-0.5218, -0.33279]    │
│ b     ┆ [-0.5218, -0.33279]    │
│ b     ┆ [-0.5218, -0.33279]    │
│ b     ┆ [-0.5218, -0.33279]    │
│ b     ┆ [-0.5218, -0.33279]    │
└───────┴────────────────────────┘

In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

shape: (5, 5)
┌──────────┬──────────┬───────────┬───────────┬───────────┐
│ x1       ┆ x2       ┆ y         ┆ pred      ┆ resid     │
│ ---      ┆ ---      ┆ ---       ┆ ---       ┆ ---       │
│ f64      ┆ f64      ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪══════════╪═══════════╪═══════════╪═══════════╡
│ 0.181184 ┆ 0.833164 ┆ -0.087106 ┆ -0.372821 ┆ 0.285715  │
│ 0.716097 ┆ 0.7239   ┆ 0.315413  ┆ -0.603674 ┆ 0.919088  │
│ 0.803562 ┆ 0.138619 ┆ -0.436603 ┆ -0.449304 ┆ 0.012702  │
│ 0.924235 ┆ 0.08284  ┆ 0.055943  ┆ -0.490844 ┆ 0.546787  │
│ 0.460823 ┆ 0.085475 ┆ -1.355857 ┆ -0.25969  ┆ -1.096167 │
└──────────┴──────────┴───────────┴───────────┴───────────┘

In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

shape: (2, 2)
┌───────┬────────────────────────┐
│ dummy ┆ coeffs                 │
│ ---   ┆ ---                    │
│ str   ┆ list[f64]              │
╞═══════╪════════════════════════╡
│ a     ┆ [-0.479674, -0.344547] │
│ b     ┆ [-0.5218, -0.33279]    │
└───────┴────────────────────────┘

In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

shape: (2, 2)
┌───────┬────────────────────────┐
│ dummy ┆ coeffs                 │
│ ---   ┆ ---                    │
│ str   ┆ list[f64]              │
╞═══════╪════════════════════════╡
│ a     ┆ [-0.299928, -0.187761] │
│ b     ┆ [-0.347887, -0.161111] │
└───────┴────────────────────────┘

In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

shape: (2, 2)
┌───────┬───────────┐
│ dummy ┆ lasso_r2  │
│ ---   ┆ ---       │
│ str   ┆ f64       │
╞═══════╪═══════════╡
│ a     ┆ -0.533955 │
│ b     ┆ -0.547336 │
└───────┴───────────┘

In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

shape: (10_000, 5)
┌───────────┬──────────┬──────────┬────────────────────────┬───────────┐
│ y         ┆ x1       ┆ x2       ┆ coeffs                 ┆ pred      │
│ ---       ┆ ---      ┆ ---      ┆ ---                    ┆ ---       │
│ f64       ┆ f64      ┆ f64      ┆ list[f64]              ┆ f64       │
╞═══════════╪══════════╪══════════╪════════════════════════╪═══════════╡
│ -0.087106 ┆ 0.181184 ┆ 0.833164 ┆ null                   ┆ null      │
│ 0.315413  ┆ 0.716097 ┆ 0.7239   ┆ null                   ┆ null      │
│ -0.436603 ┆ 0.803562 ┆ 0.138619 ┆ null                   ┆ null      │
│ 0.055943  ┆ 0.924235 ┆ 0.08284  ┆ null                   ┆ null      │
│ -1.355857 ┆ 0.460823 ┆ 0.085475 ┆ [-0.434778, 0.298689]  ┆ -0.174825 │
│ …         ┆ …        ┆ …        ┆ …                      ┆ …         │
│ -0.785811 ┆ 0.297807 ┆ 0.209961 ┆ [-1.72997, -0.058576]  ┆ -0.527496 │
│ -0.391738 ┆ 0.26132  ┆ 0.040594 ┆ [-1.856937, 0.204615]  ┆ -0.476948 │
│ -0.472705 ┆ 0.881679 ┆ 0.572239 ┆ [0.021692, -1.521368]  ┆ -0.851461 │
│ -0.41373  ┆ 0.933442 ┆ 0.189696 ┆ [-0.181844, -1.202663] ┆ -0.397881 │
│ -0.058646 ┆ 0.153629 ┆ 0.836968 ┆ [-0.610511, -0.035761] ┆ -0.123723 │
└───────────┴──────────┴──────────┴────────────────────────┴───────────┘

In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

shape: (1, 1)
┌──────┐
│ y    │
│ ---  │
│ f64  │
╞══════╡
│ -0.0 │
└──────┘

In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

shape: (1, 1)
┌─────────────────────────────────┐
│ a                               │
│ ---                             │
│ list[f64]                       │
╞═════════════════════════════════╡
│ [29.073839, 28.893157, 28.4042… │
└─────────────────────────────────┘

In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

shape: (2, 2)
┌────────────────┬───────────────────────┐
│ singular_value ┆ weight_vector         │
│ ---            ┆ ---                   │
│ f64            ┆ list[f64]             │
╞════════════════╪═══════════════════════╡
│ 29.015447      ┆ [0.568763, 0.822502]  │
│ 28.458258      ┆ [0.822502, -0.568763] │
└────────────────┴───────────────────────┘

In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

shape: (5, 1)
┌───────────┐
│ pc1       │
│ ---       │
│ f64       │
╞═══════════╡
│ 0.380626  │
│ -0.147048 │
│ -0.395586 │
│ 0.562945  │
│ 0.307737  │
└───────────┘

# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


shape: (2, 8)
┌──────────────┬──────────┬──────────┬───────────┬──────────┬──────────┬────────────────┬──────────┐
│ dummy_groups ┆ l2       ┆ log loss ┆ precision ┆ recall   ┆ f        ┆ average_precis ┆ roc_auc  │
│ ---          ┆ ---      ┆ ---      ┆ ---       ┆ ---      ┆ ---      ┆ ion            ┆ ---      │
│ str          ┆ f64      ┆ f64      ┆ f64       ┆ f64      ┆ f64      ┆ ---            ┆ f64      │
│              ┆          ┆          ┆           ┆          ┆          ┆ f64            ┆          │
╞══════════════╪══════════╪══════════╪═══════════╪══════════╪══════════╪════════════════╪══════════╡
│ a            ┆ 0.33544  ┆ 0.997945 ┆ 0.496479  ┆ 0.501581 ┆ 0.499017 ┆ 0.498435       ┆ 0.493508 │
│ b            ┆ 0.332956 ┆ 1.001033 ┆ 0.518898  ┆ 0.509274 ┆ 0.514041 ┆ 0.512584       ┆ 0.500236 │
└──────────────┴──────────┴──────────┴───────────┴──────────┴──────────┴────────────────┴──────────┘

# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

shape: (5, 2)
┌─────────────────────────────────┬───────┐
│ sen                             ┆ word  │
│ ---                             ┆ ---   │
│ str                             ┆ str   │
╞═════════════════════════════════╪═══════╡
│ Hello, world! I'm going to chu… ┆ words │
│ Hello, world! I'm going to chu… ┆ word  │
│ Hello, world! I'm going to chu… ┆ words │
│ Hello, world! I'm going to chu… ┆ word  │
│ Hello, world! I'm going to chu… ┆ words │
└─────────────────────────────────┴───────┘

In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

shape: (5, 1)
┌────────┐
│ sen    │
│ ---    │
│ str    │
╞════════╡
│ world  │
│ church │
│ hello  │
│ going  │
│ to     │
└────────┘

In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

shape: (5, 1)
┌────────┐
│ sen    │
│ ---    │
│ str    │
╞════════╡
│ hello  │
│        │
│ world  │
│ go     │
│ church │
└────────┘

In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

shape: (5, 1)
┌──────┐
│ word │
│ ---  │
│ u32  │
╞══════╡
│ 2    │
│ 1    │
│ 2    │
│ 1    │
│ 2    │
└──────┘

In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

shape: (5, 1)
┌──────┐
│ word │
│ ---  │
│ u32  │
╞══════╡
│ 2    │
│ 1    │
│ 2    │
│ 1    │
│ 2    │
└──────┘

In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

shape: (5, 1)
┌──────┐
│ word │
│ ---  │
│ f64  │
╞══════╡
│ 0.6  │
│ 0.8  │
│ 0.6  │
│ 0.8  │
│ 0.6  │
└──────┘

In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

shape: (5, 2)
┌─────────────────────────────────┬──────┐
│ sen                             ┆ word │
│ ---                             ┆ ---  │
│ str                             ┆ str  │
╞═════════════════════════════════╪══════╡
│ Hello, world! I'm going to chu… ┆ word │
│ Hello, world! I'm going to chu… ┆ word │
│ Hello, world! I'm going to chu… ┆ word │
│ Hello, world! I'm going to chu… ┆ word │
│ Hello, world! I'm going to chu… ┆ word │
└─────────────────────────────────┴──────┘

In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

shape: (1, 1)
┌────────┐
│ word   │
│ ---    │
│ str    │
╞════════╡
│ banana │
└────────┘

In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

shape: (2, 2)
┌──────────┬────────────┐
│ word     ┆ other_data │
│ ---      ┆ ---        │
│ str      ┆ i64        │
╞══════════╪════════════╡
│ asasasas ┆ 4          │
│ sasasass ┆ 5          │
└──────────┴────────────┘

In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


shape: (5, 6)
┌──────────┬───────────┬─────────────┬────────────────┬──────────────────┬─────────────────────────┐
│ asasasa  ┆ sasaaasss ┆ asdasadadfa ┆ LCS based Fuzz ┆ Optimal String   ┆ Jaro-Winkler - apples   │
│ ---      ┆ ---       ┆ ---         ┆ match - apples ┆ Alignment - app… ┆ ---                     │
│ f64      ┆ f64       ┆ f64         ┆ ---            ┆ ---              ┆ f64                     │
│          ┆           ┆             ┆ f64            ┆ f64              ┆                         │
╞══════════╪═══════════╪═════════════╪════════════════╪══════════════════╪═════════════════════════╡
│ 0.142857 ┆ 0.111111  ┆ 0.090909    ┆ 0.833333       ┆ 0.833333         ┆ 0.966667                │
│ 0.428571 ┆ 0.333333  ┆ 0.272727    ┆ 0.166667       ┆ 0.0              ┆ 0.444444                │
│ 0.111111 ┆ 0.111111  ┆ 0.090909    ┆ 0.555556       ┆ 0.444444         ┆ 0.5                     │
│ 0.875    ┆ 0.666667  ┆ 0.545455    ┆ 0.25           ┆ 0.25             ┆ 0.527778                │
│ 0.75     ┆ 0.777778  ┆ 0.454545    ┆ 0.25           ┆ 0.25             ┆ 0.527778                │
└──────────┴───────────┴─────────────┴────────────────┴──────────────────┴─────────────────────────┘

# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

shape: (5, 1)
┌───────────┐
│ a         │
│ ---       │
│ f64       │
╞═══════════╡
│ null      │
│ null      │
│ -0.667205 │
│ -0.004369 │
│ -1.539039 │
└───────────┘

In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

shape: (5, 3)
┌───────────┬───────────────┬─────────────────────────────────┐
│ a         ┆ random_normal ┆ random_normal_that_respects_nu… │
│ ---       ┆ ---           ┆ ---                             │
│ f64       ┆ f64           ┆ f64                             │
╞═══════════╪═══════════════╪═════════════════════════════════╡
│ null      ┆ 0.766622      ┆ null                            │
│ null      ┆ 0.626792      ┆ null                            │
│ -0.667205 ┆ 1.568425      ┆ -1.028465                       │
│ -0.004369 ┆ -0.846736     ┆ 1.131894                        │
│ -1.539039 ┆ -0.994406     ┆ 1.053838                        │
└───────────┴───────────────┴─────────────────────────────────┘

In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

shape: (5, 3)
┌───────────┬────────────┬─────────────────────────────────┐
│ a         ┆ random_str ┆ random_str_that_respects_null_… │
│ ---       ┆ ---        ┆ ---                             │
│ f64       ┆ str        ┆ str                             │
╞═══════════╪════════════╪═════════════════════════════════╡
│ null      ┆ FL         ┆ null                            │
│ null      ┆ ftnwd      ┆ null                            │
│ -0.667205 ┆ 7YV        ┆ o1                              │
│ -0.004369 ┆ G7         ┆ Ys                              │
│ -1.539039 ┆ Jd4        ┆ 3umWr                           │
└───────────┴────────────┴─────────────────────────────────┘

In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

shape: (5, 2)
┌───────────┬────────────┐
│ a         ┆ random_str │
│ ---       ┆ ---        │
│ f64       ┆ str        │
╞═══════════╪════════════╡
│ null      ┆ null       │
│ null      ┆ null       │
│ -0.667205 ┆ hIQx3      │
│ -0.004369 ┆ OZsZn      │
│ -1.539039 ┆ OXelh      │
└───────────┴────────────┘

In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

shape: (5, 4)
┌───────────┬───────────┬──────────┬─────────────────┐
│ a         ┆ test1     ┆ literal  ┆ test1_perturbed │
│ ---       ┆ ---       ┆ ---      ┆ ---             │
│ f64       ┆ f64       ┆ f64      ┆ f64             │
╞═══════════╪═══════════╪══════════╪═════════════════╡
│ null      ┆ -0.647906 ┆ null     ┆ -0.648101       │
│ null      ┆ 0.721174  ┆ null     ┆ 0.721425        │
│ -0.667205 ┆ 0.610471  ┆ 1.476693 ┆ 0.610372        │
│ -0.004369 ┆ -0.054558 ┆ 0.705794 ┆ -0.054194       │
│ -1.539039 ┆ 0.266183  ┆ 0.060374 ┆ 0.266557        │
└───────────┴───────────┴──────────┴─────────────────┘

In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

shape: (5, 4)
┌───────────┬──────────┬───────────┬──────────────────┐
│ a         ┆ [0, 1)   ┆ Normal    ┆ Int from [0, 10) │
│ ---       ┆ ---      ┆ ---       ┆ ---              │
│ f64       ┆ f64      ┆ f64       ┆ i32              │
╞═══════════╪══════════╪═══════════╪══════════════════╡
│ null      ┆ 0.685005 ┆ 0.366216  ┆ 8                │
│ null      ┆ 0.24877  ┆ -1.189034 ┆ 4                │
│ -0.667205 ┆ 0.89082  ┆ 0.946259  ┆ 4                │
│ -0.004369 ┆ 0.951503 ┆ 0.606895  ┆ 9                │
│ -1.539039 ┆ 0.707259 ┆ -0.907175 ┆ 7                │
└───────────┴──────────┴───────────┴──────────────────┘

In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

shape: (1, 4)
┌─────────────────────┬─────────────────┬────────────────────────────┬────────────────────────┐
│ t-tests: statistics ┆ t-tests: pvalue ┆ normality_test: statistics ┆ normality_test: pvalue │
│ ---                 ┆ ---             ┆ ---                        ┆ ---                    │
│ f64                 ┆ f64             ┆ f64                        ┆ f64                    │
╞═════════════════════╪═════════════════╪════════════════════════════╪════════════════════════╡
│ -0.941026           ┆ 0.346844        ┆ 1.420034                   ┆ 0.491636               │
└─────────────────────┴─────────────────┴────────────────────────────┴────────────────────────┘

In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

shape: (5, 5)
┌───────────┬──────────┬──────────┬────────────┬────────────┐
│ market_id ┆ var1     ┆ var2     ┆ category_1 ┆ category_2 │
│ ---       ┆ ---      ┆ ---      ┆ ---        ┆ ---        │
│ i64       ┆ f64      ┆ f64      ┆ i32        ┆ i32        │
╞═══════════╪══════════╪══════════╪════════════╪════════════╡
│ 0         ┆ 0.842972 ┆ 0.450364 ┆ 3          ┆ 4          │
│ 1         ┆ 0.625663 ┆ 0.095083 ┆ 0          ┆ 6          │
│ 2         ┆ 0.029255 ┆ 0.515528 ┆ 3          ┆ 4          │
│ 0         ┆ 0.782569 ┆ 0.664478 ┆ 1          ┆ 4          │
│ 1         ┆ 0.487103 ┆ 0.935361 ┆ 0          ┆ 7          │
└───────────┴──────────┴──────────┴────────────┴────────────┘

In [40]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

shape: (1, 3)
┌─────────────────────┬──────────────────────┬─────────────────────┐
│ t-test              ┆ chi2-test            ┆ f-test              │
│ ---                 ┆ ---                  ┆ ---                 │
│ struct[2]           ┆ struct[2]            ┆ struct[2]           │
╞═════════════════════╪══════════════════════╪═════════════════════╡
│ {1.550123,0.121144} ┆ {22.889401,0.955922} ┆ {1.746959,0.136718} │
└─────────────────────┴──────────────────────┴─────────────────────┘

In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {2.182824,0.029118}  ┆ {38.555846,0.35473}  ┆ {1.597169,0.172493} │
│ 1         ┆ {0.972503,0.330871}  ┆ {26.35273,0.880507}  ┆ {0.864623,0.484451} │
│ 2         ┆ {-0.471616,0.637232} ┆ {34.267769,0.551134} ┆ {0.783499,0.535832} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

shape: (9, 2)
┌─────────────────┬──────────────────────────┐
│ first_digit_cnt ┆ first_digit_distribution │
│ ---             ┆ ---                      │
│ u32             ┆ f64                      │
╞═════════════════╪══════════════════════════╡
│ 556             ┆ 0.1112                   │
│ 566             ┆ 0.1132                   │
│ 556             ┆ 0.1112                   │
│ 525             ┆ 0.105                    │
│ 592             ┆ 0.1184                   │
│ 520             ┆ 0.104                    │
│ 566             ┆ 0.1132                   │
│ 552             ┆ 0.1104                   │
│ 567             ┆ 0.1134                   │
└─────────────────┴──────────────────────────┘

# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

shape: (5, 7)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       ┆ nb_l_inf_cnt │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---          │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ u32          │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════════╡
│ 0   ┆ 0.347784 ┆ 0.13026  ┆ 0.334019 ┆ 0.698491 ┆ 7.909586 ┆ 15           │
│ 1   ┆ 0.48221  ┆ 0.050991 ┆ 0.736185 ┆ 0.892089 ┆ 7.823451 ┆ 8            │
│ 2   ┆ 0.786648 ┆ 0.639778 ┆ 0.774721 ┆ 0.134284 ┆ 3.51514  ┆ 20           │
│ 3   ┆ 0.944763 ┆ 0.129409 ┆ 0.460358 ┆ 0.715857 ┆ 8.133778 ┆ 16           │
│ 4   ┆ 0.597698 ┆ 0.747696 ┆ 0.885392 ┆ 0.670841 ┆ 2.392687 ┆ 19           │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────────┘

In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

shape: (5, 7)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┬─────────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       ┆ nb_l1_r_cnt │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---         │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ u32         │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╪═════════════╡
│ 0   ┆ 0.347784 ┆ 0.13026  ┆ 0.334019 ┆ 0.698491 ┆ 7.909586 ┆ 538         │
│ 1   ┆ 0.48221  ┆ 0.050991 ┆ 0.736185 ┆ 0.892089 ┆ 7.823451 ┆ 783         │
│ 2   ┆ 0.786648 ┆ 0.639778 ┆ 0.774721 ┆ 0.134284 ┆ 3.51514  ┆ 10          │
│ 3   ┆ 0.944763 ┆ 0.129409 ┆ 0.460358 ┆ 0.715857 ┆ 8.133778 ┆ 389         │
│ 4   ┆ 0.597698 ┆ 0.747696 ┆ 0.885392 ┆ 0.670841 ┆ 2.392687 ┆ 483         │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┴─────────────┘

In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

shape: (5, 7)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────────────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       ┆ best friends      │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---               │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ list[u32]         │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╪═══════════════════╡
│ 0   ┆ 0.347784 ┆ 0.13026  ┆ 0.334019 ┆ 0.698491 ┆ 7.909586 ┆ [0, 502, … 115]   │
│ 1   ┆ 0.48221  ┆ 0.050991 ┆ 0.736185 ┆ 0.892089 ┆ 7.823451 ┆ [1, 1527, … 400]  │
│ 2   ┆ 0.786648 ┆ 0.639778 ┆ 0.774721 ┆ 0.134284 ┆ 3.51514  ┆ [2, 1430, … 1451] │
│ 3   ┆ 0.944763 ┆ 0.129409 ┆ 0.460358 ┆ 0.715857 ┆ 8.133778 ┆ [3, 598, … 711]   │
│ 4   ┆ 0.597698 ┆ 0.747696 ┆ 0.885392 ┆ 0.670841 ┆ 2.392687 ┆ [4, 650, … 213]   │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┴───────────────────┘

In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 502, … 875]  ┆ 10                 │
│ 1   ┆ [1, 1527, … 400] ┆ 3                  │
│ 2   ┆ [2, 1430, … 549] ┆ 10                 │
│ 3   ┆ [3, 598, … 1768] ┆ 9                  │
│ 4   ┆ [4, 650, … 803]  ┆ 6                  │
└─────┴──────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

shape: (5, 8)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────────────┬──────────────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       ┆ idx              ┆ dist             │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---              ┆ ---              │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ list[u32]        ┆ list[f64]        │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════════════╪══════════════════╡
│ 0   ┆ 0.347784 ┆ 0.13026  ┆ 0.334019 ┆ 0.698491 ┆ 7.909586 ┆ [0, 502, … 115]  ┆ [0.0, 0.066443,  │
│     ┆          ┆          ┆          ┆          ┆          ┆                  ┆ … 0.072798]      │
│ 1   ┆ 0.48221  ┆ 0.050991 ┆ 0.736185 ┆ 0.892089 ┆ 7.823451 ┆ [1, 1527, … 400] ┆ [0.0, 0.049926,  │
│     ┆          ┆          ┆          ┆          ┆          ┆                  ┆ … 0.063975]      │
│ 2   ┆ 0.786648 ┆ 0.639778 ┆ 0.774721 ┆ 0.134284 ┆ 3.51514  ┆ [2, 1430, …      ┆ [0.0, 0.02861, … │
│     ┆          ┆          ┆          ┆          ┆          ┆ 1451]            ┆ 0.057878]        │
│ 3   ┆ 0.944763 ┆ 0.129409 ┆ 0.460358 ┆ 0.715857 ┆ 8.133778 ┆ [3, 598, … 711]  ┆ [0.0, 0.032508,  │
│     ┆          ┆          ┆          ┆          ┆          ┆                  ┆ … 0.046937]      │
│ 4   ┆ 0.597698 ┆ 0.747696 ┆ 0.885392 ┆ 0.670841 ┆ 2.392687 ┆ [4, 650, … 213]  ┆ [0.0, 0.068048,  │
│     ┆          ┆          ┆          ┆          ┆          ┆                  ┆ … 0.076969]      │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────────────┴──────────────────┘

In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

shape: (5, 6)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 0   ┆ 0.347784 ┆ 0.13026  ┆ 0.334019 ┆ 0.698491 ┆ 7.909586 │
│ 2   ┆ 0.786648 ┆ 0.639778 ┆ 0.774721 ┆ 0.134284 ┆ 3.51514  │
│ 5   ┆ 0.712633 ┆ 0.28485  ┆ 0.329133 ┆ 0.543338 ┆ 6.065003 │
│ 7   ┆ 0.405769 ┆ 0.443343 ┆ 0.892205 ┆ 0.731708 ┆ 9.658069 │
│ 10  ┆ 0.836991 ┆ 0.428517 ┆ 0.404204 ┆ 0.440019 ┆ 4.264234 │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┘

In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

shape: (5, 6)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 61  ┆ 0.457586 ┆ 0.507503 ┆ 0.614296 ┆ 0.55833  ┆ 7.064511 │
│ 164 ┆ 0.550585 ┆ 0.564562 ┆ 0.917242 ┆ 0.756173 ┆ 8.670795 │
│ 257 ┆ 0.571268 ┆ 0.54402  ┆ 0.110826 ┆ 0.16714  ┆ 8.900079 │
│ 334 ┆ 0.551334 ┆ 0.458245 ┆ 0.40399  ┆ 0.494347 ┆ 1.872597 │
│ 352 ┆ 0.535394 ┆ 0.459457 ┆ 0.467295 ┆ 0.986702 ┆ 3.193573 │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┘

In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

shape: (5, 6)
┌─────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ id  ┆ var1     ┆ var2     ┆ var3     ┆ r        ┆ rh       │
│ --- ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ u32 ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 61  ┆ 0.457586 ┆ 0.507503 ┆ 0.614296 ┆ 0.55833  ┆ 7.064511 │
│ 354 ┆ 0.442961 ┆ 0.519426 ┆ 0.691972 ┆ 0.94437  ┆ 7.440443 │
│ 406 ┆ 0.520402 ┆ 0.443565 ┆ 0.015572 ┆ 0.814672 ┆ 9.903239 │
│ 411 ┆ 0.523037 ┆ 0.569513 ┆ 0.975117 ┆ 0.163414 ┆ 8.785701 │
│ 488 ┆ 0.4547   ┆ 0.453289 ┆ 0.388635 ┆ 0.391255 ┆ 9.463455 │
└─────┴──────────┴──────────┴──────────┴──────────┴──────────┘

In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

shape: (5, 3)
┌─────┬──────────────────┬───────┐
│ id  ┆ friends          ┆ count │
│ --- ┆ ---              ┆ ---   │
│ u64 ┆ list[u32]        ┆ u32   │
╞═════╪══════════════════╪═══════╡
│ 0   ┆ [0, 459, … 1058] ┆ 6     │
│ 1   ┆ [1]              ┆ 1     │
│ 2   ┆ [2, 1077]        ┆ 2     │
│ 3   ┆ [3, 104]         ┆ 2     │
│ 4   ┆ [4, 781, … 650]  ┆ 4     │
└─────┴──────────────────┴───────┘

# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures.

In [53]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_28864/3354819425.py:3: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


shape: (5, 6)
┌────────┬───────────┬─────┬─────┬─────┬─────┐
│ actual ┆ predicted ┆ 0-2 ┆ 0-9 ┆ s1  ┆ s2  │
│ ---    ┆ ---       ┆ --- ┆ --- ┆ --- ┆ --- │
│ f64    ┆ f64       ┆ i32 ┆ i32 ┆ str ┆ str │
╞════════╪═══════════╪═════╪═════╪═════╪═════╡
│ 1.0    ┆ 0.965653  ┆ 2   ┆ 6   ┆ I0  ┆ nR  │
│ 0.0    ┆ 0.44037   ┆ 2   ┆ 2   ┆ 6   ┆ Mz  │
│ 1.0    ┆ 0.931955  ┆ 1   ┆ 2   ┆ 1   ┆ kg  │
│ 0.0    ┆ 0.558197  ┆ 1   ┆ 4   ┆ R6  ┆ m   │
│ 1.0    ┆ 0.235535  ┆ 2   ┆ 1   ┆ bF  ┆ RO  │
└────────┴───────────┴─────┴─────┴─────┴─────┘

In [54]:
df_pd = df.to_pandas()

In [55]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

shape: (1,)
Series: 'jaccard_col' [f64]
[
	0.3
]

In [56]:
# Polars Series
print(pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest())
# Pyarrow Series
print(pds2.query_binary_metrics(df["actual"].to_arrow(), df["predicted"].to_arrow()).struct.unnest())
# NumPy
print(pds2.query_binary_metrics(df["actual"].to_numpy(), df["predicted"].to_numpy()).struct.unnest())

shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.499669  ┆ 0.499599 ┆ 0.499634 ┆ 0.497488          ┆ 0.498677 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.499669  ┆ 0.499599 ┆ 0.499634 ┆ 0.497488          ┆ 0.498677 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬───────

In [57]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

shape: (5, 4)
┌──────────┬──────────────┬────────────┬──────────┐
│ <=       ┆ baseline_pct ┆ actual_pct ┆ psi_bin  │
│ ---      ┆ ---          ┆ ---        ┆ ---      │
│ f64      ┆ f64          ┆ f64        ┆ f64      │
╞══════════╪══════════════╪════════════╪══════════╡
│ 0.211304 ┆ 0.2          ┆ 0.215      ┆ 0.001085 │
│ 0.399705 ┆ 0.2          ┆ 0.195      ┆ 0.000127 │
│ 0.594605 ┆ 0.2          ┆ 0.187      ┆ 0.000874 │
│ 0.797354 ┆ 0.2          ┆ 0.189      ┆ 0.000622 │
│ inf      ┆ 0.2          ┆ 0.214      ┆ 0.000947 │
└──────────┴──────────────┴────────────┴──────────┘

In [58]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

shape: (1,)
Series: 'cid_ce' [f64]
[
	13.128145
]

In [59]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

shape: (1,)
Series: 'c3_stats' [f64]
[
	0.11619
]

In [60]:
pds2.str_leven(df["s1"], df["s2"]).head() 

shape: (10,)
Series: 'str_leven' [u32]
[
	2
	2
	2
	2
	2
	2
	2
	1
	2
	2
]